Installing dependencies

In [1]:
%pip install pandas
%pip install --upgrade ipykernel
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Importing dependencies

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

Reading CSVs

In [3]:
train = pd.read_csv('input/train.csv', index_col='id').reset_index(drop=True)
test = pd.read_csv('input/test.csv', index_col='id').reset_index(drop=True)
sample_submission = pd.read_csv('input/submission_sample.csv')
meta = pd.read_csv('input/metadata.csv')

Getting columns type

In [21]:
cat_nom = [x for x in meta.iloc[1:-1, :].loc[(meta.iloc[:,1]=="Qualitativo nominal")].iloc[:,0]]
cat_ord = [x for x in meta.iloc[1:-1, :].loc[(meta.iloc[:,1]=="Qualitativo ordinal")].iloc[:,0]]
num_dis = [x for x in meta.iloc[1:-1, :].loc[(meta.iloc[:,1]=="Quantitativo discreto")].iloc[:,0]]
num_con = [x for x in meta.iloc[1:-1, :].loc[(meta.iloc[:,1]=="Quantitativo continua")].iloc[:,0]]

Splitting testing values

In [22]:
X = train.drop(columns=['y'], axis=1)
y = train['y']

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.1, random_state=15)

Pre-processing data

In [23]:

pipeline_quantitativo = Pipeline([
    ('scaler', StandardScaler())
])

pipeline_qualitativo = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')) 
])

transformador = ColumnTransformer([
    ('quantitativo', pipeline_quantitativo, num_con+num_dis),
    ('qualitativo', pipeline_qualitativo, cat_ord+cat_nom)
], remainder='passthrough')

pipeline_completo = Pipeline([
    ('preprocessamento', transformador),
    ('classificador', GradientBoostingClassifier(random_state=15))
])

Training model with data

In [24]:
pipeline_completo.fit(X_treino, y_treino)

Pipeline(steps=[('preprocessamento',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('quantitativo',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['var55', 'var56', 'var57',
                                                   'var58', 'var59', 'var60',
                                                   'var61', 'var62', 'var63',
                                                   'var64', 'var65', 'var66',
                                                   'var24', 'var25', 'var27',
                                                   'var40', 'var44', 'var45',
                                                   'var46', 'var47', 'var48',
                                                   'var49', 'var50', 'var51',
                                                   'va...
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['var26', 'var32', 'var42',
                                                   'var43', 'var1', 'var2',
                                                   'var3', 'var4', 'var5',
                                                   'var6', 'var7', 'var8',
                                                   'var9', 'var10', 'var11',
                                                   'var12', 'var13', 'var14',
                                                   'var15', 'var16', 'var17',
                                                   'var18', 'var19', 'var20',
                                                   'var21', 'var22', 'var23',
                                                   'var28', 'var29', 'var30', ...])])),
                ('classificador', GradientBoostingClassifier(random_state=15))])

Getting prediction using train data

In [8]:
previsoes = pipeline_completo.predict(X_teste)

Show model precision

In [13]:
precisao = accuracy_score(y_teste, previsoes)
print(f"Precisão do modelo: {precisao*100:.2f}%")

Precisão do modelo: 84.01%


Getting prediction using test data

In [14]:
previsoes_classes = pipeline_completo.predict(test)

probabilidades = pipeline_completo.predict_proba(test)

Show results

In [15]:
for i in range(len(test)):
    print(f"Linha {i+1}:")
    print(f"   - Maior probabilidade: {previsoes_classes[i]}")
    print(f"   - Probabilidade de 0: {probabilidades[i][0]:.2f}")
    print(f"   - Probabilidade de 1: {probabilidades[i][1]:.2f}")

Linha 1:
   - Maior probabilidade: 1
   - Probabilidade de 0: 0.44
   - Probabilidade de 1: 0.56
Linha 2:
   - Maior probabilidade: 1
   - Probabilidade de 0: 0.28
   - Probabilidade de 1: 0.72
Linha 3:
   - Maior probabilidade: 0
   - Probabilidade de 0: 0.86
   - Probabilidade de 1: 0.14
Linha 4:
   - Maior probabilidade: 1
   - Probabilidade de 0: 0.40
   - Probabilidade de 1: 0.60
Linha 5:
   - Maior probabilidade: 1
   - Probabilidade de 0: 0.42
   - Probabilidade de 1: 0.58
Linha 6:
   - Maior probabilidade: 0
   - Probabilidade de 0: 0.90
   - Probabilidade de 1: 0.10
Linha 7:
   - Maior probabilidade: 1
   - Probabilidade de 0: 0.31
   - Probabilidade de 1: 0.69
Linha 8:
   - Maior probabilidade: 0
   - Probabilidade de 0: 0.94
   - Probabilidade de 1: 0.06
Linha 9:
   - Maior probabilidade: 1
   - Probabilidade de 0: 0.17
   - Probabilidade de 1: 0.83
Linha 10:
   - Maior probabilidade: 0
   - Probabilidade de 0: 0.97
   - Probabilidade de 1: 0.03
Linha 11:
   - Maior probabil